# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import string
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df.drop(columns = ['id','original', 'message', 'genre'])

In [3]:
#check for nulls in X and Y
print(X.isnull().sum())
print(Y.isnull().sum()/Y.shape[0])
print('X shape is {}, Y shape is {}'.format(X.shape,Y.shape))

0
related                   0.005238
request                   0.005238
offer                     0.005238
aid_related               0.005238
medical_help              0.005238
medical_products          0.005238
search_and_rescue         0.005238
security                  0.005238
military                  0.005238
child_alone               0.005238
water                     0.005238
food                      0.005238
shelter                   0.005238
clothing                  0.005238
money                     0.005238
missing_people            0.005238
refugees                  0.005238
death                     0.005238
other_aid                 0.005238
infrastructure_related    0.005238
transport                 0.005238
buildings                 0.005238
electricity               0.005238
tools                     0.005238
hospitals                 0.005238
shops                     0.005238
aid_centers               0.005238
other_infrastructure      0.005238
weather_related   

In [4]:
# drop NAs in the Y
Y = Y.dropna(how = 'all', axis = 0)
#drop same rows in X
X = X[0:Y.shape[0]]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text, lemmatizer = WordNetLemmatizer()):
        """
        Returns a normalized, lemmatized list of tokens from a document by
        applying segmentation (breaking into sentences), then word/punctuation
        tokenization, and finally part of speech tagging. It uses the part of
        speech tags to look up the lemma in WordNet, and returns the lowercase
        version of all the words, removing stopwords and punctuation.
        """
        tokens = []
        # Break the document into sentences
        for sent in sent_tokenize(text):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(word_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() 
                token = token.strip()
                token = token.strip('_')
                token = token.strip('*')

                # If punctuation or stopword, ignore token and continue
                if token in set(sw.words('english')) or all(char in string.punctuation for char in token):
                    continue
                
                tokens.append(token)
        # Lemmatize the token and yield
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
        return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline= Pipeline([
            ('vectorizer', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('classifier', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=3))),
        ])

In [7]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vectorizer', 'tfidf', 'classifier', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'classifier__estimator__algorithm', 'classifier__estimator__leaf_size', 'classifier__estimator__metric', 'classifier__estimator__metric_params', 'classifier__estimator__n_jobs', 'classifier__estimator__n_neighbors', 'classifier__estimator__p', 'classifier__estimator__weights', 'classifier__estimator', 'classifier__n_jobs'])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.65, random_state = 33)

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred  = pipeline.predict(X_test)

In [11]:
def report(y_pred,y_original):
    i = 0
    for item in y_original.columns:
        print(classification_report(y_original[item],y_pred[:,i]))
        i+=1

In [12]:
report(y_pred,y_test)

             precision    recall  f1-score   support

        0.0       0.25      0.45      0.32      3920
        1.0       0.78      0.57      0.66     12999
        2.0       0.08      0.15      0.10       116

avg / total       0.65      0.54      0.58     17035

             precision    recall  f1-score   support

        0.0       0.84      0.95      0.89     14146
        1.0       0.33      0.12      0.17      2889

avg / total       0.75      0.81      0.77     17035

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     16956
        1.0       0.00      0.00      0.00        79

avg / total       0.99      1.00      0.99     17035

             precision    recall  f1-score   support

        0.0       0.59      0.83      0.69      9977
        1.0       0.42      0.17      0.24      7058

avg / total       0.52      0.56      0.50     17035

             precision    recall  f1-score   support

        0.0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
'classifier__estimator__n_neighbors':list(range(3,10)),
'classifier__estimator__weights':['uniform', 'distance']
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=3)

cv

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__n_neighbors': [3, 4, 5, 6, 7, 8, 9], 'classifier__estimator__weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train,y_train)
print(cv.best_params_)

Fitting 2 folds for each of 14 candidates, totalling 28 fits
[CV] classifier__estimator__n_neighbors=3, classifier__estimator__weights=uniform 
[CV]  classifier__estimator__n_neighbors=3, classifier__estimator__weights=uniform, score=0.12690798081116442, total= 1.5min
[CV] classifier__estimator__n_neighbors=3, classifier__estimator__weights=uniform 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


[CV]  classifier__estimator__n_neighbors=3, classifier__estimator__weights=uniform, score=0.17466201482773658, total= 1.5min
[CV] classifier__estimator__n_neighbors=3, classifier__estimator__weights=distance 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  4.8min remaining:    0.0s


[CV]  classifier__estimator__n_neighbors=3, classifier__estimator__weights=distance, score=0.12516354121238552, total= 1.4min
[CV] classifier__estimator__n_neighbors=3, classifier__estimator__weights=distance 
[CV]  classifier__estimator__n_neighbors=3, classifier__estimator__weights=distance, score=0.10052333187963367, total= 1.4min
[CV] classifier__estimator__n_neighbors=4, classifier__estimator__weights=uniform 
[CV]  classifier__estimator__n_neighbors=4, classifier__estimator__weights=uniform, score=0.17204535542956825, total= 1.6min
[CV] classifier__estimator__n_neighbors=4, classifier__estimator__weights=uniform 
[CV]  classifier__estimator__n_neighbors=4, classifier__estimator__weights=uniform, score=0.19930222416048846, total= 1.5min
[CV] classifier__estimator__n_neighbors=4, classifier__estimator__weights=distance 
[CV]  classifier__estimator__n_neighbors=4, classifier__estimator__weights=distance, score=0.1190580026166594, total= 1.5min
[CV] classifier__estimator__n_neighbors

[Parallel(n_jobs=-1)]: Done  28 out of  28 | elapsed: 68.7min finished


{'classifier__estimator__n_neighbors': 4, 'classifier__estimator__weights': 'uniform'}


In [15]:
cv_pred  = cv.predict(X_test)

In [16]:
cv_report = report(cv_pred,y_test)


             precision    recall  f1-score   support

        0.0       0.27      0.22      0.24      3920
        1.0       0.77      0.82      0.80     12999
        2.0       0.08      0.11      0.10       116

avg / total       0.65      0.67      0.66     17035

             precision    recall  f1-score   support

        0.0       0.83      0.99      0.90     14146
        1.0       0.38      0.04      0.07      2889

avg / total       0.76      0.83      0.76     17035

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     16956
        1.0       0.00      0.00      0.00        79

avg / total       0.99      1.00      0.99     17035

             precision    recall  f1-score   support

        0.0       0.59      0.93      0.72      9977
        1.0       0.43      0.07      0.12      7058

avg / total       0.52      0.58      0.47     17035

             precision    recall  f1-score   support

        0.0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# try a different classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])
pipeline2.fit(X_train, y_train)
y_pred2  = pipeline2.predict(X_test)

In [18]:
report(y_pred2,y_test)

             precision    recall  f1-score   support

        0.0       0.30      0.00      0.01      3920
        1.0       0.76      0.99      0.86     12999
        2.0       0.03      0.03      0.03       116

avg / total       0.65      0.76      0.66     17035

             precision    recall  f1-score   support

        0.0       0.84      0.96      0.90     14146
        1.0       0.38      0.12      0.18      2889

avg / total       0.76      0.82      0.78     17035

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     16956
        1.0       0.00      0.00      0.00        79

avg / total       0.99      0.99      0.99     17035

             precision    recall  f1-score   support

        0.0       0.59      0.86      0.70      9977
        1.0       0.46      0.16      0.24      7058

avg / total       0.54      0.57      0.51     17035

             precision    recall  f1-score   support

        0.0       0.92      1.00 

In [19]:
pipeline2.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [20]:
parameters2 = {'clf__estimator__learning_rate': [0.1, 0.3],
            'clf__estimator__n_estimators': [10, 50, 100]
             }
cv2 = GridSearchCV(pipeline2, param_grid=parameters2, cv=2, n_jobs=-1, verbose=3)

cv2

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.3], 'clf__estimator__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [21]:
cv2.fit(X_train,y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.17531617967727867, total= 1.2min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=10, score=0.19624945486262538, total= 1.2min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.4min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.18054949847361534, total= 1.5min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, score=0.19188835586567815, total= 1.5min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.17488006977758394, total= 1.9min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.18229393807239425, total= 1.9min
[CV] clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=10, score=0.18229393807239425, total= 1.2min
[CV] clf__estimator__learning_rate=0.3, clf__estimator__n_estimators=10 
[CV]  clf__estimator__learning_rate=0.3, clf__estimator_

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 24.8min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__learning_rate': [0.1, 0.3], 'clf__estimator__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [22]:
print(cv2.best_params_)

{'clf__estimator__learning_rate': 0.3, 'clf__estimator__n_estimators': 10}


In [23]:
cv2_pred = cv2.predict(X_test)
cv2_report = report(cv2_pred,y_test)

             precision    recall  f1-score   support

        0.0       0.08      0.00      0.00      3920
        1.0       0.76      1.00      0.87     12999
        2.0       0.00      0.00      0.00       116

avg / total       0.60      0.76      0.66     17035

             precision    recall  f1-score   support

        0.0       0.83      1.00      0.91     14146
        1.0       0.33      0.01      0.01      2889

avg / total       0.75      0.83      0.76     17035

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     16956
        1.0       0.00      0.00      0.00        79

avg / total       0.99      1.00      0.99     17035

             precision    recall  f1-score   support

        0.0       0.59      1.00      0.74      9977
        1.0       0.35      0.00      0.01      7058

avg / total       0.49      0.58      0.43     17035

             precision    recall  f1-score   support

        0.0       0.92      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
#create a list of best for cv and cv2
#compare f1score of two cv models (2th value from the end)
run1 = 0
run2 = 0
i = 0
for item in y_test.columns:
        if classification_report(y_test[item],cv_pred[:,i]).split( )[-2]> classification_report(y_test[item],cv2_pred[:,i]).split( )[-2]:
            run1 +=1
        else: run2+=1
if run1>run2: print('cv is better performing than cv2')
else: print('cv2 is the best solution')
    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


cv is better performing than cv2


### 9. Export your model as a pickle file

In [30]:
with open('model.pkl', 'wb') as f:
    if run1>run2:    pickle.dump(cv, f)
    else:  pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

### Note to self:
    To further improve the quality of the output, the set needs to be more balanced out. With some classes more prevalent than the others, the prediction can be skewed